# Chapter 15 전이학습

## 15.1 전이학습이란?

전이학습은 미리 학습된 가중치를 이용해서 신경망 가중치를 더욱 쉽게 최적화하는 것이다.

자연어처리 분야에서도 전이학습이 주류로 자리잡고 있다.

비슷한 특징을 갖고 있는 데이터셋으로, 비슷한 문제를 해결하고자 한다면, 기존에 훈련된 가중치 파라미터를 약간씩 수정하는 것이 더욱 효과적일 수 있다. 그리고 사전 훈련된 데이터셋이 매우 방대하다면, 새로운 문제를 위한 데이터셋이 담지 못한 정보를 통해서 더 좋은 해결 능력을 얻을 수 있을 것이다.

## 15.2 기존의 사전 훈련 방식

word2vec 이후에는 사전 훈련된 단어 임베딩 벡터를 사용해서 딥러닝 모델을 개선하고자 했다.

텍스트 분류 신경망의 경우
- 임베딩, RNN, 소프트맥스 계층
- 입력은 원핫 벡터

### 15.2.1 사전 훈련된 가중치 파라미터의 사용 방법

#### 신경망 가중치 파라미터 초깃값으로 사용

#### 신경망 가중치 파라미터 값으로 고정

임베딩 계층의 가중치 파라미터를 제외한 다른 계층의 파라미터만 학습하도록 한다. 파이토치의 ```model.parameters()```를 이용해서 임베딩 계층을 제외한 파라미터들만 반환한다.

#### 신경망 가중치 파라미터 초깃값으로 사용 및 느린 학습

임베딩 계층을 동결하는 것이 아닌 더 작은 학습률로 천천히 학습하도록 할 수 있다.

### 15.2.2 사전 훈련된 단어 임베딩 벡터를 쓰지 않는 이유

#### 문장의 문맥을 반영하지 못하는 기존 단어 임베딩 알고리즘

Skip-gram이나 GloVe 알고리즘은 단어를 잠재 공간에 성공적으로 임베딩할 수 있었지만 문장에서 함께 출현한 단어들을 예측하는데 기반한다. 임베딩된 정보는 매우 한정적이고, 목적 함수 또한 실제 우리가 풀려는 문제와 상이할 것이다. 또한, 단어는 문장의 문맥에 따라 의미가 확연히 달라지는데 기존 단어 임베딩은 지나치게 단순하다.

#### 신경망의 입력 계층의 가중치 파라미터에만 적용

임베딩 계층의 전역 최소점에 가까울지라도 전체 신경망 가중치의 전역 최소값일지는 알 수 없다.

### 15.2.3 자연어 처리에서 전이학습의 효과

일반 문장들로 언어 모델을 훈련하는 것만으로도 전이학습으로 인한 성능 향상을 볼 수 있다. 일반 문장의 경우 데이터 수집 비용이 굉장히 싼 편이다. 이러한 점으로 자연어 처리 분야에서 사전훈련이 다른 분야의 사전훈련에 비해 훨씬 유리하다.

## 15.3 ELMo

입력을 단방향 LSTM에 넣고, 정방향과 역방향 언어모델을 각각 훈련한다. 정방향, 역방향 LSTM은 따로 구성되지만 임베딩 계층과 소프트맥스 계층은 가중치 파라미터를 공유한다.

$$\mathcal L (\Theta) = \sum_{k=1}^N log p(t_k|t_1, \dots, t_{k-1}; \theta_x, \overrightarrow {\theta}_{LSTM}, \theta_s) + log p(t_{k+1}, \dots, t_N; \theta_x, \overleftarrow {\theta}_{LSTM}, \theta_s)$$
$$where \ \Theta = \{\theta_x, \overrightarrow {\theta}_{LSTM}, \overleftarrow {\theta}_{LSTM}, \theta_s \}$$

계층별 은닉상태를 time-step별로 얻고, 가중합을 구하여, 해당 단어(토큰)에 대한 문장 내에서의 임베딩 표현인 $ELMo_k^{task}$를 구할 수 있다.

$$ELMo_k^task = \gamma^{task} \sum_{j=0}^L s_j^{task} \cdot h_{k, j}^LM$$
$$where h_{k,j}^LM = 
\begin{cases}
x_k & \text{if }j = 0, \\
[\overrightarrow {h}_{k,j}^{LM}; \overleftarrow {h}_{k,j}^{LM}] & \text{if }j > 0.
\end{cases}$$

사전 훈련된 ELMo 벡터는 입력 또는 출력 벡터와 연계된다. ELMo 표현을 위한 신경망 가중치 파라미터는 갱신되지 않는다. LSTM의 정방향과 역방향 은닉 상태를 포함하므로 문맥에 따른 단어의 의미도 표현할 수 있다.

SQuAD, SRL, NER 등 다양한 자연어 처리 문제에서 전이학습을 수행할 수 있다.

ELMo는 신경망의 은닉 상태를 임베딩에 사용함으로써 문장의 문맥에 따른 다양한 의미를 반영할 수 있다. 게다가 레이블링이 필요없는 일반 문장들을 통해 사전 훈련함으로써 성능을 크게 끌어올릴 수 있다.

## 15.4 BERT

BERT는 파인 튜닝 기반 전이학습이다. 약간의 계층을 추가하여 학습함으로써 추가적인 노력이 적게 들어도 특정 문제에 맞는 신경망을 만들 수 있다. 

트랜스포머 모델에서 인코더만 가져와 언어 모델을 훈련하는데 사용한다. 마스크된 언어 모델링 방법인 MLM을 통해 자기 회귀 속성으로 안한 제약을 뛰어넘어, 양방향 시퀀셜 모델링을 성공적으로 수행할 수 있다. 두개의 문장을 동시에 학습하도록 함으로써 질의응답과 같은 문제에서 성공적인 예측이 가능하다.

### 15.4.1 MLM

언어 모델의 훈련을 빈칸 채우기 문제로 바꿔 양방향 인코딩을 적용할 수 있게 했다. MLM은 랜덤하게 문장 내 단어(토큰)을 MASK 토큰이나 임의의 다른 단어로 치환한 후, 원래 문장을 맞추도록 학습시킨다. MLM 도입으로 훈련시 수렴이 늦어지는 문제가 발생하긴 하지만, 그보다 더 큰 성능 개선이 이뤄졌다.

### 15.4.2 다음 문장 예측하기

문장 순서를 나타내는 정보를 추가로 임베딩하여 기존의 단어 임베딩 및 위치 임베딩의 합에 더해준다. [SEP]라는 문장의 경계를 나타내는 특수한 토큰도 사용된다. 이런 식으로 입력을 주고 다음 뭊장인지 예측하도록 훈련함으로써 문장과 문장간의 관계를 익히도록 한다. 이러한 단순한 방식으로도 질의응답 문제 해결에 큰 기여를 한다.

BERT에서는 사전 훈련 이후, 문제마다 추가 훈련을 할 때 요구되는 입력과 출력, 추가 가중치 파라미터가 달라진다. 추가된 가중치들은 기존 가중치와 함께 학습된다.

### 15.4.3 문장의 쌍 분류 문제

QNLI는 질문과 대답 관계인지 맞추는 이진 분류 문제이다.

분류 문제에 적용하기 위해서, 트랜스포머 인코더의 가장 첫 time-step 입력은 [CLS]라는 특별 토큰을 입력을 받는다. 출력 벡터에 추가 소프트맥스 계층을 추가하여 파인 튜닝함으로써 주어진 문제에 대한 분류 신경망을 얻을 수 있다.

[CLS]에 해당하는 트랜스포머 인코더의 출력 벡터를 $C \in R^H$라고 한다. 소프트맥스 계층의 가중치 파라미터 $W \in R^{K \times H}$를 추가한다. $K$는 분류 문제에서 후보 클래스의 개수를 의미한다. 이러면 각 클래스에 속할 확률 분포 $P \in R^K$를 구할 수 있다.

$$P = softmax(CW^T)$$

MLE 수행하면서 기존과 새로운 가중치 파라미터를 한꺼번에 학습한다.

### 15.4.4 단일 문장 분류 문제

[CLS] 토큰의 트랜스포머 출력에 대해 소프트맥스 계층을 추가해서 파인 튜닝해서 신경망을 훈련한다. 이렇게 얻은 신경망은 무수한 코퍼스로 훈련해서, 특정 문제에 대한 데이터셋으로만 얻어낼 수 있는 성능을 상회하는 성능을 달성할 수 있다.

### 15.4.5 질의응답 문제

질문과 문장이 주어졌을 때 문장 내에서 정답 단어 또는 구절이 위치한 time-step의 시작과 끝을 출력으로 반환한다.

- SQuAD
- KorSQuAD

### 15.4.6 단일 문장 태깅 문제

입력으로 들어온 각 단어(토큰)에 대한 태깅 정보를 출력으로 반환한다.

- NER
- SRL